In [1]:
import sqlite3
import pandas as pd
from IPython.core.display import HTML, Markdown

In [2]:
class Report:
    def __init__(self):
        self.md = ""

    def addTitle(self, title):
        self.md += f"# {title}\n\n"

    def addSubTitle(self, title):
        self.md += f"## {title}\n\n"

    def addParagraph(self, text):
        for line in text.splitlines():
            self.md += f"{line}\n"
        self.md += "\n"

    def addText(self, text):
        self.addParagraph(text)
    
    def append(self, text):
        self.md += text + "\n"

    def clear(self):
        self.md = ""

    def debug(self):
        print(self.md)

    def display(self):
        display(Markdown(self.md))

    def markdown(self):
        return self.md

    def saveToFile(self, fn):
        with open(fn, 'w') as f:
            f.write(self.md)

    def addLessonsTab(self, e, con):
        sql = f"""SELECT
                    giorno,
                    L.data as dataISO,
                    substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) as data,
                    L.idFasciaOraria,
                    L.idLezione
                FROM
                    Edizione E INNER JOIN 
                    Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
                    Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
                WHERE L.idEdizione = '{e}'
                ORDER BY E.idEdizione, L.data, L.idFasciaOraria"""
        ls = pd.read_sql_query(sql, con)
        str = "| Incontro | Data | Ora | Incontro | Data | Ora |\n|--:|:-:|:-:|--:|:-:|:-:|\n"
        for j in range(len(ls)):
            l = ls.iloc[j]
            str += f"|{l["idLezione"]}|{l["giorno"].lower()} {l["data"]} |{l["idFasciaOraria"]}"
            if j % 2 == 0:
                str += ""
            else:
                str += "|\n"
        self.md += str + "\n\n"

    def addLessonsList(self, e, con):
        sql = f"""SELECT
                    giorno,
                    L.data as dataISO,
                    substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) as data,
                    L.idFasciaOraria,
                    L.idLezione
                FROM
                    Edizione E INNER JOIN 
                    Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
                    Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
                WHERE L.idEdizione = '{e}'
                ORDER BY E.idEdizione, L.data, L.idFasciaOraria"""
        ls = pd.read_sql_query(sql, con)
        str = "\n"
        for j in range(len(ls)):
            l = ls.iloc[j]
            str += f"{l["idLezione"]}. {l["giorno"].lower()} {l["data"]} {l["idFasciaOraria"]}<sup>a</sup> ora\n"
        self.md += str + "\n\n"
        
    def addLessonsByStaff(self, p, con):
        sql = f"""SELECT
                rank() OVER (ORDER BY L.data, L.idFasciaOraria) AS progressivo,
            	E.idClasse,
            	giorno,
            	L.data AS dataISO,
            	L.idFasciaOraria,
            	E.idCorso ,
            	L.idLezione,
            	F.ruolo,
            	substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) as data
            FROM
            	Formatore F INNER JOIN
            	Personale P ON F.idPersonale=P.idPersonale INNER JOIN
            	Edizione E ON F.idEdizione=E.idEdizione INNER JOIN
            	Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
            	Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
            WHERE
            	P.idPersonale = '{p}'
            ORDER BY
            	L.data,
            	L.idFasciaOraria"""
        ls = pd.read_sql_query(sql, con)
        str = "|Progr.| Data | Ora | Corso | Lezione | Classe |\n|--:|:-:|:-:|:-:|--:|:-:|\n"
        for j in range(len(ls)):
            l = ls.iloc[j]
            str += f"|{l["progressivo"]}|{l["giorno"].lower()} {l["data"]}|{l["idFasciaOraria"]}|{l["idCorso"]}|{l["idLezione"]}|{l["idClasse"]}|\n"    
        self.md += str + "\n\n"

    def addDailySchedule(self, p, con):
        sql = f"""SELECT
				rank() OVER (ORDER BY L.data) AS progressivo,
            	giorno,
            	substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) AS data,
				group_concat(L.idFasciaOraria ORDER BY L.idFasciaOraria ASC) AS ore,
				group_concat(DISTINCT E.idCorso) AS corsi
            FROM
            	Formatore F INNER JOIN
            	Personale P ON F.idPersonale=P.idPersonale INNER JOIN
            	Edizione E ON F.idEdizione=E.idEdizione INNER JOIN
            	Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
            	Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
            WHERE
            	P.idPersonale = '{p}'
			GROUP BY L.data
            ORDER BY
            	L.data"""
        str = "|Giornata| Data | Ore | Corsi |\n|--:|:-:|:-:|:-:|\n"
        ds = pd.read_sql_query(sql, con)
        for j in range(len(ds)):
            d = ds.iloc[j]
            str += f"|{d["progressivo"]}|{d["giorno"].lower()} {d["data"]}|{d["ore"]}|{d["corsi"]}|\n"
        self.md += str + "\n\n"

In [3]:
con = sqlite3.connect("data/calendario_corsi.sqlite")
report = Report()

In [4]:
report.clear()
report.addTitle("Calendario corsi")
for c in ['RelAx', 'CyberSec', 'CryptoWithBlocks', 'LabMat2', 'PhyComp', 'RicOp']: # , 'EduRobot', 'PassInf']:
    sql = "SELECT idEdizione, denominazione, descrizione, idClasse FROM Edizione INNER JOIN Corso USING (idCorso) WHERE Corso.idCorso='" + c + "' ORDER BY denominazione"
    edizioni = pd.read_sql_query(sql, con)
    for i in range(len(edizioni)):
        edizione = edizioni.iloc[i]
        # Formatore
        sql = f"SELECT idPersonale FROM Formatore WHERE idEdizione = '{edizione["idEdizione"]}' AND ruolo = 'Formatore'"
        formatore = pd.read_sql_query(sql, con)["idPersonale"][0]
        # Tutor
        sql = f"SELECT idPersonale FROM Formatore WHERE idEdizione = '{edizione["idEdizione"]}' AND ruolo = 'Tutor'"
        tutor = pd.read_sql_query(sql, con)["idPersonale"][0]

        sql = f"""SELECT 
                	lower(giorno) ||
                	' ' || substr(L.data, 9, 2) ||
                	'/' || substr(L.data, 6, 2) ||
                	'/' || substr(L.data, 1, 4) AS avvio
                FROM
                	Lezione L INNER JOIN 
                	Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
                WHERE
                	idEdizione = '{edizione["idEdizione"]}' AND idLezione = 1"""
        dataAvvio = pd.read_sql_query(sql, con)["avvio"][0]
        sql = f"""SELECT elenco FROM ObiettiviFormativi WHERE idCorso='{c}'"""
        obiettivi = pd.read_sql_query(sql, con)["elenco"][0]
        
        editionRep = Report()

        # Titolo
        editionRep.addSubTitle(edizione["denominazione"]) # edizione["idEdizione"] + " - " + 
        
        ## Destinatari
        editionRep.addText(f"""Agli studenti della classe {edizione["idClasse"]}<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore {formatore}<br />
Al tutor {tutor}<br />
Alla segreteria didattica<br />
All\'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />""")
        
        editionRep.addText(f'**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "{edizione["denominazione"]}"**')

        editionRep.addText(f"Si comunica che, a partire dal giorno {dataAvvio}, prenderanno avvio le attività del corso “**{edizione["denominazione"]}**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.")

        # Lezioni
        editionRep.addText(f"Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: ")
        # editionRep.addLessonsTab(edizione["idEdizione"], con)
        editionRep.addLessonsList(edizione["idEdizione"], con)
        # Studenti
        #editionRep.addText(f"**Studenti/Classe**: {edizione["idClasse"]}")
        # Formatore
        #editionRep.addText(f"**Formatore**: *{formatore}*")
        # Tutor
        #editionRep.addText(f"**Tutor**: *{tutor}*")
        # Descrizione
        #editionRep.addParagraph(f"**Descrizione**: {edizione["descrizione"]}")
        editionRep.addParagraph(edizione["descrizione"])
        editionRep.addParagraph("""Gli obiettivi generali del corso includono:""")
        editionRep.addParagraph(obiettivi)
        
        
        editionRep.addText("Docente referente del progetto: Prof. Gionata Massi.\n\nCordiali saluti.\n\nAncona, xx.01.2025")
        editionRep.saveToFile("./gen_docs/editions/" + edizione["idEdizione"] + ".md")
        report.append(editionRep.markdown())
report.display()

# Calendario corsi

## ALGEBRA DELLE RELAZIONI 1

Agli studenti della classe S5B+S5D<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore Massi<br />
Al tutor Pomili<br />
Alla segreteria didattica<br />
All'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />

**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "ALGEBRA DELLE RELAZIONI 1"**

Si comunica che, a partire dal giorno ven 13/12/2024, prenderanno avvio le attività del corso “**ALGEBRA DELLE RELAZIONI 1**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.

Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: 


1. ven 13/12/2024 5<sup>a</sup> ora
2. ven 20/12/2024 5<sup>a</sup> ora
3. ven 10/01/2025 5<sup>a</sup> ora
4. ven 17/01/2025 5<sup>a</sup> ora
5. ven 24/01/2025 5<sup>a</sup> ora
7. ven 21/03/2025 5<sup>a</sup> ora
9. ven 04/04/2025 5<sup>a</sup> ora
10. ven 11/04/2025 5<sup>a</sup> ora
11. ven 09/05/2025 5<sup>a</sup> ora
12. ven 09/05/2025 6<sup>a</sup> ora


Il percorso introduce le basi della teoria delle basi di dati relazionali, un pilastro fondamentale dell'informatica intesa come disciplina che studia i metodi per estrarre informazioni in modo automatico da dati grezzi.
La presentazione degli argomenti è declinata in modo rendere evidente come l'informatica sia una disciplina scientifica e ingegneristica. Vengono esposti sia gli aspetti scientifici e formali, come quelli dell'algebra delle relazioni, sia quelli ingegneristici legati all'efficienza computazionale degli algoritmi.
I contenuti si collocano in un contesto interdisciplinare, come è proprio dell'informatica, dove la teoria si unisce alla pratica, grazie all'utilizzo di una piattaforma web per la valutazione di espressioni algebriche relazionali.
Obiettivi generali:
  -  cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
  - comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
  - utilizzare strumenti informatici per modellare e risolvere problemi.
  - acquisire una comprensione approfondita della disciplina informatica per orientare le proprie scelte future, sia a livello universitario che professionale.

Gli obiettivi generali del corso includono:

- Comprendre meno superficialmente gli oggetti di studio e i metodi della disciplina informatica per poter orientare com maggior consapevolezza le proprie scelte future.
- Stimolare l’interesse verso le materie tecnico scientifiche e, in particolare, verso l’informatica.
- Cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
- Comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
- Utilizzare strumenti informatici per modellizzare e risolvere problemi.

Docente referente del progetto: Prof. Gionata Massi.

Cordiali saluti.

Ancona, xx.01.2025


## ALGEBRA DELLE RELAZIONI 2

Agli studenti della classe S4E<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore Massi<br />
Al tutor Pomili<br />
Alla segreteria didattica<br />
All'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />

**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "ALGEBRA DELLE RELAZIONI 2"**

Si comunica che, a partire dal giorno mer 05/02/2025, prenderanno avvio le attività del corso “**ALGEBRA DELLE RELAZIONI 2**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.

Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: 


1. mer 05/02/2025 2<sup>a</sup> ora
2. mer 12/02/2025 2<sup>a</sup> ora
3. mer 19/02/2025 2<sup>a</sup> ora
4. mer 26/02/2025 2<sup>a</sup> ora
5. mer 05/03/2025 2<sup>a</sup> ora
6. mer 12/03/2025 2<sup>a</sup> ora
7. mer 19/03/2025 2<sup>a</sup> ora
8. mer 26/03/2025 2<sup>a</sup> ora
9. mer 02/04/2025 2<sup>a</sup> ora
10. mer 09/04/2025 2<sup>a</sup> ora


Il percorso introduce le basi della teoria delle basi di dati relazionali, un pilastro fondamentale dell'informatica intesa come disciplina che studia i metodi per estrarre informazioni in modo automatico da dati grezzi.
La presentazione degli argomenti è declinata in modo rendere evidente come l'informatica sia una disciplina scientifica e ingegneristica. Vengono esposti sia gli aspetti scientifici e formali, come quelli dell'algebra delle relazioni, sia quelli ingegneristici legati all'efficienza computazionale degli algoritmi.
I contenuti si collocano in un contesto interdisciplinare, come è proprio dell'informatica, dove la teoria si unisce alla pratica, grazie all'utilizzo di una piattaforma web per la valutazione di espressioni algebriche relazionali.
Obiettivi generali:
  -  cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
  - comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
  - utilizzare strumenti informatici per modellare e risolvere problemi.
  - acquisire una comprensione approfondita della disciplina informatica per orientare le proprie scelte future, sia a livello universitario che professionale.

Gli obiettivi generali del corso includono:

- Comprendre meno superficialmente gli oggetti di studio e i metodi della disciplina informatica per poter orientare com maggior consapevolezza le proprie scelte future.
- Stimolare l’interesse verso le materie tecnico scientifiche e, in particolare, verso l’informatica.
- Cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
- Comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
- Utilizzare strumenti informatici per modellizzare e risolvere problemi.

Docente referente del progetto: Prof. Gionata Massi.

Cordiali saluti.

Ancona, xx.01.2025


## ALGEBRA DELLE RELAZIONI 3

Agli studenti della classe S5E<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore Massi<br />
Al tutor Sciamanna<br />
Alla segreteria didattica<br />
All'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />

**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "ALGEBRA DELLE RELAZIONI 3"**

Si comunica che, a partire dal giorno mer 11/12/2024, prenderanno avvio le attività del corso “**ALGEBRA DELLE RELAZIONI 3**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.

Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: 


1. mer 11/12/2024 3<sup>a</sup> ora
2. lun 16/12/2024 2<sup>a</sup> ora
3. mer 18/12/2024 3<sup>a</sup> ora
4. mer 08/01/2025 3<sup>a</sup> ora
5. lun 13/01/2025 2<sup>a</sup> ora
6. mer 15/01/2025 3<sup>a</sup> ora
7. lun 20/01/2025 2<sup>a</sup> ora
8. mer 22/01/2025 3<sup>a</sup> ora
9. mer 29/01/2025 3<sup>a</sup> ora
10. mer 05/02/2025 3<sup>a</sup> ora


Il percorso introduce le basi della teoria delle basi di dati relazionali, un pilastro fondamentale dell'informatica intesa come disciplina che studia i metodi per estrarre informazioni in modo automatico da dati grezzi.
La presentazione degli argomenti è declinata in modo rendere evidente come l'informatica sia una disciplina scientifica e ingegneristica. Vengono esposti sia gli aspetti scientifici e formali, come quelli dell'algebra delle relazioni, sia quelli ingegneristici legati all'efficienza computazionale degli algoritmi.
I contenuti si collocano in un contesto interdisciplinare, come è proprio dell'informatica, dove la teoria si unisce alla pratica, grazie all'utilizzo di una piattaforma web per la valutazione di espressioni algebriche relazionali.
Obiettivi generali:
  -  cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
  - comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
  - utilizzare strumenti informatici per modellare e risolvere problemi.
  - acquisire una comprensione approfondita della disciplina informatica per orientare le proprie scelte future, sia a livello universitario che professionale.

Gli obiettivi generali del corso includono:

- Comprendre meno superficialmente gli oggetti di studio e i metodi della disciplina informatica per poter orientare com maggior consapevolezza le proprie scelte future.
- Stimolare l’interesse verso le materie tecnico scientifiche e, in particolare, verso l’informatica.
- Cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
- Comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
- Utilizzare strumenti informatici per modellizzare e risolvere problemi.

Docente referente del progetto: Prof. Gionata Massi.

Cordiali saluti.

Ancona, xx.01.2025


## ALGEBRA DELLE RELAZIONI 4

Agli studenti della classe S4F<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore Massi<br />
Al tutor Sciamanna<br />
Alla segreteria didattica<br />
All'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />

**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "ALGEBRA DELLE RELAZIONI 4"**

Si comunica che, a partire dal giorno mer 11/12/2024, prenderanno avvio le attività del corso “**ALGEBRA DELLE RELAZIONI 4**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.

Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: 


1. mer 11/12/2024 4<sup>a</sup> ora
2. lun 16/12/2024 1<sup>a</sup> ora
3. mer 18/12/2024 4<sup>a</sup> ora
4. mer 08/01/2025 4<sup>a</sup> ora
5. lun 13/01/2025 1<sup>a</sup> ora
6. mer 15/01/2025 4<sup>a</sup> ora
7. lun 20/01/2025 1<sup>a</sup> ora
8. mer 22/01/2025 4<sup>a</sup> ora
9. mer 29/01/2025 4<sup>a</sup> ora
10. mer 26/02/2025 4<sup>a</sup> ora


Il percorso introduce le basi della teoria delle basi di dati relazionali, un pilastro fondamentale dell'informatica intesa come disciplina che studia i metodi per estrarre informazioni in modo automatico da dati grezzi.
La presentazione degli argomenti è declinata in modo rendere evidente come l'informatica sia una disciplina scientifica e ingegneristica. Vengono esposti sia gli aspetti scientifici e formali, come quelli dell'algebra delle relazioni, sia quelli ingegneristici legati all'efficienza computazionale degli algoritmi.
I contenuti si collocano in un contesto interdisciplinare, come è proprio dell'informatica, dove la teoria si unisce alla pratica, grazie all'utilizzo di una piattaforma web per la valutazione di espressioni algebriche relazionali.
Obiettivi generali:
  -  cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
  - comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
  - utilizzare strumenti informatici per modellare e risolvere problemi.
  - acquisire una comprensione approfondita della disciplina informatica per orientare le proprie scelte future, sia a livello universitario che professionale.

Gli obiettivi generali del corso includono:

- Comprendre meno superficialmente gli oggetti di studio e i metodi della disciplina informatica per poter orientare com maggior consapevolezza le proprie scelte future.
- Stimolare l’interesse verso le materie tecnico scientifiche e, in particolare, verso l’informatica.
- Cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
- Comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
- Utilizzare strumenti informatici per modellizzare e risolvere problemi.

Docente referente del progetto: Prof. Gionata Massi.

Cordiali saluti.

Ancona, xx.01.2025


## CYBER SICUREZZA 1

Agli studenti della classe S3B<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore Taccari<br />
Al tutor Sciamanna<br />
Alla segreteria didattica<br />
All'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />

**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "CYBER SICUREZZA 1"**

Si comunica che, a partire dal giorno mer 29/01/2025, prenderanno avvio le attività del corso “**CYBER SICUREZZA 1**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.

Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: 


1. mer 29/01/2025 5<sup>a</sup> ora
2. ven 31/01/2025 6<sup>a</sup> ora
3. mer 05/02/2025 5<sup>a</sup> ora
5. mer 12/02/2025 5<sup>a</sup> ora
6. ven 14/02/2025 6<sup>a</sup> ora
7. mer 19/02/2025 5<sup>a</sup> ora
8. mer 05/03/2025 5<sup>a</sup> ora
9. ven 07/03/2025 6<sup>a</sup> ora
10. mer 12/03/2025 5<sup>a</sup> ora
12. mer 19/03/2025 5<sup>a</sup> ora
13. ven 21/03/2025 6<sup>a</sup> ora
14. mer 26/03/2025 5<sup>a</sup> ora
15. ven 28/03/2025 6<sup>a</sup> ora
16. mer 02/04/2025 5<sup>a</sup> ora
17. ven 04/04/2025 6<sup>a</sup> ora
18. mer 09/04/2025 5<sup>a</sup> ora
19. ven 11/04/2025 6<sup>a</sup> ora
20. mer 16/04/2025 5<sup>a</sup> ora
21. mer 23/04/2025 5<sup>a</sup> ora
22. mer 30/04/2025 5<sup>a</sup> ora


0

Gli obiettivi generali del corso includono:

- Diventare consapevoli delle problematiche di sicurezza nell'uso di strumenti e tecnologie informatiche
- Possedere le conoscenze di base necessarie a operare in sicurezza nel digitale, con responsabile attenzione per la sicurezza dei dati personali
- Rafforzare le competenze nell’affrontare i rischi legati all’uso di social network, al trattamento dei dati e alle truffe online.
- Stimolare l’interesse verso le materie tecnico scientifiche e, in particolare, verso l’informatica.
- Cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
- Comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
- Utilizzare strumenti informatici per modellizzare e risolvere problemi.

Docente referente del progetto: Prof. Gionata Massi.

Cordiali saluti.

Ancona, xx.01.2025


## CYBER SICUREZZA 2

Agli studenti della classe S4B<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore Taccari<br />
Al tutor Sciamanna<br />
Alla segreteria didattica<br />
All'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />

**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "CYBER SICUREZZA 2"**

Si comunica che, a partire dal giorno mer 29/01/2025, prenderanno avvio le attività del corso “**CYBER SICUREZZA 2**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.

Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: 


1. mer 29/01/2025 6<sup>a</sup> ora
2. ven 31/01/2025 4<sup>a</sup> ora
5. ven 14/02/2025 4<sup>a</sup> ora
6. mer 19/02/2025 6<sup>a</sup> ora
10. mer 05/03/2025 6<sup>a</sup> ora
11. ven 07/03/2025 4<sup>a</sup> ora
12. mer 12/03/2025 6<sup>a</sup> ora
14. mer 19/03/2025 6<sup>a</sup> ora
15. ven 21/03/2025 4<sup>a</sup> ora
16. mer 26/03/2025 6<sup>a</sup> ora
17. ven 28/03/2025 4<sup>a</sup> ora
18. mer 02/04/2025 6<sup>a</sup> ora
19. ven 04/04/2025 4<sup>a</sup> ora
20. mer 09/04/2025 6<sup>a</sup> ora
21. ven 11/04/2025 4<sup>a</sup> ora
22. mer 16/04/2025 6<sup>a</sup> ora
24. mer 23/04/2025 6<sup>a</sup> ora
26. mer 30/04/2025 6<sup>a</sup> ora
28. mer 07/05/2025 6<sup>a</sup> ora
29. ven 09/05/2025 4<sup>a</sup> ora


0

Gli obiettivi generali del corso includono:

- Diventare consapevoli delle problematiche di sicurezza nell'uso di strumenti e tecnologie informatiche
- Possedere le conoscenze di base necessarie a operare in sicurezza nel digitale, con responsabile attenzione per la sicurezza dei dati personali
- Rafforzare le competenze nell’affrontare i rischi legati all’uso di social network, al trattamento dei dati e alle truffe online.
- Stimolare l’interesse verso le materie tecnico scientifiche e, in particolare, verso l’informatica.
- Cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
- Comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
- Utilizzare strumenti informatici per modellizzare e risolvere problemi.

Docente referente del progetto: Prof. Gionata Massi.

Cordiali saluti.

Ancona, xx.01.2025


## CRITTOGRAFIA CON BLOCCHI 1

Agli studenti della classe S1B<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore Taccari<br />
Al tutor Sciamanna<br />
Alla segreteria didattica<br />
All'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />

**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "CRITTOGRAFIA CON BLOCCHI 1"**

Si comunica che, a partire dal giorno mer 05/02/2025, prenderanno avvio le attività del corso “**CRITTOGRAFIA CON BLOCCHI 1**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.

Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: 


1. mer 05/02/2025 1<sup>a</sup> ora
2. sab 08/02/2025 4<sup>a</sup> ora
3. mer 12/02/2025 1<sup>a</sup> ora
4. sab 15/02/2025 4<sup>a</sup> ora
5. mer 19/02/2025 1<sup>a</sup> ora
6. sab 22/02/2025 4<sup>a</sup> ora
8. sab 01/03/2025 4<sup>a</sup> ora
9. mer 05/03/2025 1<sup>a</sup> ora
10. sab 08/03/2025 4<sup>a</sup> ora
11. mer 12/03/2025 1<sup>a</sup> ora


Il corso fornisce un'introduzione alle "grandi idee" della crittografia tramite un approccio didattico che procede "per scoperta" e “per necessità”. Partendo da attività progettate per la comprensione dei crittosistemi classici si procede per tappe fino alla crittografia moderna. Ogni attività prevede la sperimentazione del crittosistema, una realizzazione di quelle grandi idee, e richiede agli studenti la costruzione di possibili procedure di attacco. Gli studenti saranno quindi in grado di valutare i limiti del sistema stesso e la consapevolezza delle debolezze dovrebbe indurre la necessità di scoprire un crittosistema successivo  (approccio didattico orientato dalla necessità, Necessity Learning Design).
Il corso si basa sul lavoro di M. Lodi - M. Sbaraglia - S. Martini, "PROGRAMMARE PER IMPARARE LA CRITTOGRAFIA AL LICEO MATEMATICO" e usa il materiale didattico realizzato in Snap! (un linguaggio di programmazione visuale a blocchi) da Michael Lodi (Università di Bologna) che consiste in procedure di codifica e decodifica, di analisi e di attacco e attività guidate

Gli obiettivi generali del corso includono:

- Diventare consapevoli delle problematiche di sicurezza nell'uso di strumenti e tecnologie informatiche
- Possedere le conoscenze di base necessarie a operare in sicurezza nel digitale, con responsabile attenzione per la sicurezza dei dati personali
- Rafforzare le competenze nell’affrontare i rischi legati all’uso di social network, al trattamento dei dati e alle truffe online.
- Stimolare l’interesse verso le materie tecnico scientifiche e, in particolare, verso l’informatica.
- Cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
- Comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
- Utilizzare strumenti informatici per modellizzare e risolvere problemi.

Docente referente del progetto: Prof. Gionata Massi.

Cordiali saluti.

Ancona, xx.01.2025


## CRITTOGRAFIA CON BLOCCHI 2

Agli studenti della classe S4G<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore Taccari<br />
Al tutor Lorenzoni<br />
Alla segreteria didattica<br />
All'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />

**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "CRITTOGRAFIA CON BLOCCHI 2"**

Si comunica che, a partire dal giorno mar 04/02/2025, prenderanno avvio le attività del corso “**CRITTOGRAFIA CON BLOCCHI 2**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.

Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: 


1. mar 04/02/2025 2<sup>a</sup> ora
2. mar 11/02/2025 2<sup>a</sup> ora
3. mar 18/02/2025 2<sup>a</sup> ora
5. mar 04/03/2025 2<sup>a</sup> ora
6. mar 11/03/2025 2<sup>a</sup> ora
7. mar 18/03/2025 2<sup>a</sup> ora
8. mar 25/03/2025 2<sup>a</sup> ora
9. mar 01/04/2025 2<sup>a</sup> ora
10. mar 08/04/2025 2<sup>a</sup> ora
11. mar 15/04/2025 2<sup>a</sup> ora


Il corso fornisce un'introduzione alle "grandi idee" della crittografia tramite un approccio didattico che procede "per scoperta" e “per necessità”. Partendo da attività progettate per la comprensione dei crittosistemi classici si procede per tappe fino alla crittografia moderna. Ogni attività prevede la sperimentazione del crittosistema, una realizzazione di quelle grandi idee, e richiede agli studenti la costruzione di possibili procedure di attacco. Gli studenti saranno quindi in grado di valutare i limiti del sistema stesso e la consapevolezza delle debolezze dovrebbe indurre la necessità di scoprire un crittosistema successivo  (approccio didattico orientato dalla necessità, Necessity Learning Design).
Il corso si basa sul lavoro di M. Lodi - M. Sbaraglia - S. Martini, "PROGRAMMARE PER IMPARARE LA CRITTOGRAFIA AL LICEO MATEMATICO" e usa il materiale didattico realizzato in Snap! (un linguaggio di programmazione visuale a blocchi) da Michael Lodi (Università di Bologna) che consiste in procedure di codifica e decodifica, di analisi e di attacco e attività guidate

Gli obiettivi generali del corso includono:

- Diventare consapevoli delle problematiche di sicurezza nell'uso di strumenti e tecnologie informatiche
- Possedere le conoscenze di base necessarie a operare in sicurezza nel digitale, con responsabile attenzione per la sicurezza dei dati personali
- Rafforzare le competenze nell’affrontare i rischi legati all’uso di social network, al trattamento dei dati e alle truffe online.
- Stimolare l’interesse verso le materie tecnico scientifiche e, in particolare, verso l’informatica.
- Cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
- Comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
- Utilizzare strumenti informatici per modellizzare e risolvere problemi.

Docente referente del progetto: Prof. Gionata Massi.

Cordiali saluti.

Ancona, xx.01.2025


## CRITTOGRAFIA CON LINGUAGGIO A BLOCCHI

Agli studenti della classe S2B<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore Massi<br />
Al tutor Pomili<br />
Alla segreteria didattica<br />
All'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />

**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "CRITTOGRAFIA CON LINGUAGGIO A BLOCCHI"**

Si comunica che, a partire dal giorno mer 05/02/2025, prenderanno avvio le attività del corso “**CRITTOGRAFIA CON LINGUAGGIO A BLOCCHI**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.

Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: 


1. mer 05/02/2025 1<sup>a</sup> ora
2. mer 05/02/2025 4<sup>a</sup> ora
3. mer 12/02/2025 4<sup>a</sup> ora
4. mer 19/02/2025 4<sup>a</sup> ora
5. mer 05/03/2025 4<sup>a</sup> ora
6. mer 12/03/2025 4<sup>a</sup> ora
7. mer 19/03/2025 4<sup>a</sup> ora
9. mer 02/04/2025 4<sup>a</sup> ora
10. mer 09/04/2025 4<sup>a</sup> ora
11. mer 16/04/2025 4<sup>a</sup> ora


Il corso fornisce un'introduzione alle "grandi idee" della crittografia tramite un approccio didattico che procede "per scoperta" e “per necessità”. Partendo da attività progettate per la comprensione dei crittosistemi classici si procede per tappe fino alla crittografia moderna. Ogni attività prevede la sperimentazione del crittosistema, una realizzazione di quelle grandi idee, e richiede agli studenti la costruzione di possibili procedure di attacco. Gli studenti saranno quindi in grado di valutare i limiti del sistema stesso e la consapevolezza delle debolezze dovrebbe indurre la necessità di scoprire un crittosistema successivo  (approccio didattico orientato dalla necessità, Necessity Learning Design).
Il corso si basa sul lavoro di M. Lodi - M. Sbaraglia - S. Martini, "PROGRAMMARE PER IMPARARE LA CRITTOGRAFIA AL LICEO MATEMATICO" e usa il materiale didattico realizzato in Snap! (un linguaggio di programmazione visuale a blocchi) da Michael Lodi (Università di Bologna) che consiste in procedure di codifica e decodifica, di analisi e di attacco e attività guidate

Gli obiettivi generali del corso includono:

- Diventare consapevoli delle problematiche di sicurezza nell'uso di strumenti e tecnologie informatiche
- Possedere le conoscenze di base necessarie a operare in sicurezza nel digitale, con responsabile attenzione per la sicurezza dei dati personali
- Rafforzare le competenze nell’affrontare i rischi legati all’uso di social network, al trattamento dei dati e alle truffe online.
- Stimolare l’interesse verso le materie tecnico scientifiche e, in particolare, verso l’informatica.
- Cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
- Comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
- Utilizzare strumenti informatici per modellizzare e risolvere problemi.

Docente referente del progetto: Prof. Gionata Massi.

Cordiali saluti.

Ancona, xx.01.2025


## LABORATORIO DI MATEMATICA 2

Agli studenti della classe LicMat<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore Massi<br />
Al tutor Marrucchiello<br />
Alla segreteria didattica<br />
All'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />

**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "LABORATORIO DI MATEMATICA 2"**

Si comunica che, a partire dal giorno ven 31/01/2025, prenderanno avvio le attività del corso “**LABORATORIO DI MATEMATICA 2**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.

Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: 


1. ven 31/01/2025 5<sup>a</sup> ora
2. ven 31/01/2025 6<sup>a</sup> ora
3. ven 07/02/2025 5<sup>a</sup> ora
4. ven 07/02/2025 6<sup>a</sup> ora
5. ven 14/02/2025 5<sup>a</sup> ora
6. ven 14/02/2025 6<sup>a</sup> ora
7. ven 28/02/2025 5<sup>a</sup> ora
8. ven 28/02/2025 6<sup>a</sup> ora
9. ven 07/03/2025 5<sup>a</sup> ora
10. ven 07/03/2025 6<sup>a</sup> ora


Il corso mira a sviluppare il pensiero ricorsivo attraverso lo sviluppo dell'aritmetica. Tra induzione e ricorsione saranno definiti l'insieme dei numeri naturali e, con le sole funzioni primitive di riconoscimento dello zero, successore e predecessore, saranno definiti alcuni operatori di confronto e operazioni di addizione, sottrazione, moltiplicazione e divisione intera. Lo stesso modo di ragionare su problemi numerici può essere applicato alla risoluzione di tutti problemi che l'informatica può risolvere. Sono questi ragionamenti il vero oggetto del corso.

Gli obiettivi generali del corso includono:

- Comprendre meno superficialmente gli oggetti di studio e i metodi della disciplina informatica per poter orientare com maggior consapevolezza le proprie scelte future.
- Stimolare l’interesse verso le materie tecnico scientifiche e, in particolare, verso l’informatica.
- Cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
- Comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
- Utilizzare strumenti informatici per modellizzare e risolvere problemi.

Docente referente del progetto: Prof. Gionata Massi.

Cordiali saluti.

Ancona, xx.01.2025


## PHYSICAL COMPUTING

Agli studenti della classe E2B<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore Massi<br />
Al tutor Sciamanna<br />
Alla segreteria didattica<br />
All'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />

**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "PHYSICAL COMPUTING"**

Si comunica che, a partire dal giorno lun 10/03/2025, prenderanno avvio le attività del corso “**PHYSICAL COMPUTING**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.

Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: 


1. lun 10/03/2025 2<sup>a</sup> ora
2. lun 17/03/2025 2<sup>a</sup> ora
3. lun 24/03/2025 2<sup>a</sup> ora
4. lun 31/03/2025 2<sup>a</sup> ora
5. lun 07/04/2025 2<sup>a</sup> ora
6. lun 14/04/2025 2<sup>a</sup> ora
8. mer 23/04/2025 4<sup>a</sup> ora
9. lun 28/04/2025 2<sup>a</sup> ora
10. mer 30/04/2025 4<sup>a</sup> ora
11. mer 07/05/2025 4<sup>a</sup> ora


Il percorso vuole fornire un'introduzione al Physical computing e usando Micro:bit, Raspberry Pi e Arduino. L'uso delle schede elettroniche programmabili permette l'interazione con il mondo reale e rende visibile l'effetto dell'esecuzione del programma che lo studente andrà a realizzare. Obiettivi del corso sono la 

Gli obiettivi generali del corso includono:

- Comprendre meno superficialmente gli oggetti di studio e i metodi della disciplina informatica per poter orientare com maggior consapevolezza le proprie scelte future.
- Stimolare l’interesse verso le materie tecnico scientifiche e, in particolare, verso l’informatica.
- Cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
- Comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
- Utilizzare strumenti informatici per modellizzare e risolvere problemi.

Docente referente del progetto: Prof. Gionata Massi.

Cordiali saluti.

Ancona, xx.01.2025


## RICERCA OPERATIVA

Agli studenti della classe E5ASIA<br />
Alle loro famiglie<br />
Ai loro insegnanti<br />
Al formatore Marinelli<br />
Al tutor Massi<br />
Alla segreteria didattica<br />
All'Ufficio personale<br />
Ai collaboratori scolastici<br />
Al DSGA M. Zagaglia<br />

**Oggetto: PTOF 2024-25 Avvio corso DM65/23 Linea A edizione "RICERCA OPERATIVA"**

Si comunica che, a partire dal giorno lun 03/02/2025, prenderanno avvio le attività del corso “**RICERCA OPERATIVA**”. Il percorso rientra nell’ambito del progetto scolastico “Citizen scientists of the future” M4C1I3.1-2023-1143-P-31174, C.U.P. H34D23002330006 finanziato dal Piano nazionale di ripresa e resilienza, Missione 4 – Istruzione e ricerca – Componente 1 – Potenziamento dell’offerta dei servizi di istruzione: dagli asili nido alle università – Investimento 3.1 “Nuove competenze e nuovi linguaggi”, finanziato dall’Unione europea – Next Generation EU – “Azioni di potenziamento delle competenze STEM e multilinguistiche”.

Le lezioni saranno svolte, salvo imprevisti, nelle date seguenti: 


1. lun 03/02/2025 1<sup>a</sup> ora
2. lun 03/02/2025 2<sup>a</sup> ora
3. lun 10/02/2025 1<sup>a</sup> ora
4. lun 10/02/2025 2<sup>a</sup> ora
5. lun 17/02/2025 1<sup>a</sup> ora
6. lun 17/02/2025 2<sup>a</sup> ora
7. lun 24/02/2025 1<sup>a</sup> ora
8. lun 24/02/2025 2<sup>a</sup> ora
9. lun 03/03/2025 1<sup>a</sup> ora
10. lun 03/03/2025 2<sup>a</sup> ora


Il corso, articolato in 10 ore e rivolto agli studenti delle classi quinte, intende offrire un’esperienza formativa rivolta principalmente alla comprensione del valore pratico della matematica, attraverso lo sviluppo di competenze di problem solving scientifico e un approccio critico alla realtà.
Uno degli aspetti centrali del corso è insegnare a risolvere problemi complessi in modo sistematico, incoraggiando gli studenti a riflettere sui dati, sui metodi e sui criteri che stanno alla base delle soluzioni proposte. Questo esercizio favorisce lo sviluppo del senso critico e, dal punto di vista delle competenze, della capacità di modellare problemi reali attraverso strumenti matematici, qualità indispensabili per valutare l’efficacia e la robustezza delle scelte decisionali in ogni contesto applicativo.
Il corso illustra metodi e strumenti a cavallo tra matematica e informatica. La Ricerca Operativa sarà utilizzata per mostrare il ruolo complementare delle due discipline: da un lato, la matematica come strumento per costruire modelli e analizzarne le proprietà; dall’altro, l’informatica come mezzo per sviluppare algoritmi e implementare soluzioni. Questa sinergia non solo arricchisce il bagaglio culturale degli studenti, offrendo una visione interdisciplinare dove la matematica non è un fine a sé stessa, ma getta nuova luce sull’importanza di discipline tradizionali come algebra, geometria e analisi.
Infine, l’obiettivo generale del corso è accrescere la motivazione allo studio della matematica, specialmente per quegli studenti che spesso incontrano difficoltà e percepiscono questa disciplina come arida o distante. Attraverso esempi concreti e significativi, il corso vuole mostrare come la matematica possa essere uno strumento utile e appassionante, contribuendo così a ridurre il divario di rendimento tra gli studenti.

Gli obiettivi generali del corso includono:

- Comprendre meno superficialmente gli oggetti di studio e i metodi della disciplina informatica per poter orientare com maggior consapevolezza le proprie scelte future.
- Stimolare l’interesse verso le materie tecnico scientifiche e, in particolare, verso l’informatica.
- Cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
- Comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
- Utilizzare strumenti informatici per modellizzare e risolvere problemi.

Docente referente del progetto: Prof. Gionata Massi.

Cordiali saluti.

Ancona, xx.01.2025




In [5]:
report.clear()
# Orario formatori
report.addTitle("Orario del personale")
for p in ['Lorenzoni', 'Massi', 'Pomili', 'Sciamanna', 'Taccari']:
    staffLessonsRep = Report()
    staffLessonsRep.addSubTitle(f"{p} - Lezioni")
    staffLessonsRep.addLessonsByStaff(p, con)
    staffLessonsRep.saveToFile("./gen_docs/lessons/" + p.lower() + ".md")
    report.append(staffLessonsRep.markdown())

report.display()

# Orario del personale

## Lorenzoni - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|
|1|mar 04/02/2025|2|CryptoWithBlocks|1|S4G|
|2|mar 11/02/2025|2|CryptoWithBlocks|2|S4G|
|3|mar 18/02/2025|2|CryptoWithBlocks|3|S4G|
|4|mar 04/03/2025|2|CryptoWithBlocks|5|S4G|
|5|mar 11/03/2025|2|CryptoWithBlocks|6|S4G|
|6|mar 18/03/2025|2|CryptoWithBlocks|7|S4G|
|7|mar 25/03/2025|2|CryptoWithBlocks|8|S4G|
|8|mar 01/04/2025|2|CryptoWithBlocks|9|S4G|
|9|mar 08/04/2025|2|CryptoWithBlocks|10|S4G|
|10|mar 15/04/2025|2|CryptoWithBlocks|11|S4G|



## Massi - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|
|1|mer 11/12/2024|3|RelAx|1|S5E|
|2|mer 11/12/2024|4|RelAx|1|S4F|
|3|ven 13/12/2024|5|RelAx|1|S5B+S5D|
|4|lun 16/12/2024|1|RelAx|2|S4F|
|5|lun 16/12/2024|2|RelAx|2|S5E|
|6|mer 18/12/2024|3|RelAx|3|S5E|
|7|mer 18/12/2024|4|RelAx|3|S4F|
|8|ven 20/12/2024|5|RelAx|2|S5B+S5D|
|9|mer 08/01/2025|3|RelAx|4|S5E|
|10|mer 08/01/2025|4|RelAx|4|S4F|
|11|ven 10/01/2025|5|RelAx|3|S5B+S5D|
|12|lun 13/01/2025|1|RelAx|5|S4F|
|13|lun 13/01/2025|2|RelAx|5|S5E|
|14|mer 15/01/2025|3|RelAx|6|S5E|
|15|mer 15/01/2025|4|RelAx|6|S4F|
|16|ven 17/01/2025|5|RelAx|4|S5B+S5D|
|17|lun 20/01/2025|1|RelAx|7|S4F|
|18|lun 20/01/2025|2|RelAx|7|S5E|
|19|mer 22/01/2025|3|RelAx|8|S5E|
|20|mer 22/01/2025|4|RelAx|8|S4F|
|21|ven 24/01/2025|5|RelAx|5|S5B+S5D|
|22|mer 29/01/2025|3|RelAx|9|S5E|
|23|mer 29/01/2025|4|RelAx|9|S4F|
|24|ven 31/01/2025|5|LabMat2|1|LicMat|
|25|ven 31/01/2025|6|LabMat2|2|LicMat|
|26|lun 03/02/2025|1|RicOp|1|E5ASIA|
|27|lun 03/02/2025|2|RicOp|2|E5ASIA|
|28|mer 05/02/2025|1|CryptoWithBlocks|1|S2B|
|29|mer 05/02/2025|2|RelAx|1|S4E|
|30|mer 05/02/2025|3|RelAx|10|S5E|
|31|mer 05/02/2025|4|CryptoWithBlocks|2|S2B|
|32|ven 07/02/2025|5|LabMat2|3|LicMat|
|33|ven 07/02/2025|6|LabMat2|4|LicMat|
|34|lun 10/02/2025|1|RicOp|3|E5ASIA|
|35|lun 10/02/2025|2|RicOp|4|E5ASIA|
|36|mer 12/02/2025|2|RelAx|2|S4E|
|37|mer 12/02/2025|4|CryptoWithBlocks|3|S2B|
|38|ven 14/02/2025|5|LabMat2|5|LicMat|
|39|ven 14/02/2025|6|LabMat2|6|LicMat|
|40|lun 17/02/2025|1|RicOp|5|E5ASIA|
|41|lun 17/02/2025|2|RicOp|6|E5ASIA|
|42|mer 19/02/2025|2|RelAx|3|S4E|
|43|mer 19/02/2025|4|CryptoWithBlocks|4|S2B|
|44|lun 24/02/2025|1|RicOp|7|E5ASIA|
|45|lun 24/02/2025|2|RicOp|8|E5ASIA|
|46|mer 26/02/2025|2|RelAx|4|S4E|
|47|mer 26/02/2025|4|RelAx|10|S4F|
|48|ven 28/02/2025|5|LabMat2|7|LicMat|
|49|ven 28/02/2025|6|LabMat2|8|LicMat|
|50|lun 03/03/2025|1|RicOp|9|E5ASIA|
|51|lun 03/03/2025|2|RicOp|10|E5ASIA|
|52|mer 05/03/2025|2|RelAx|5|S4E|
|53|mer 05/03/2025|4|CryptoWithBlocks|5|S2B|
|54|ven 07/03/2025|5|LabMat2|9|LicMat|
|55|ven 07/03/2025|6|LabMat2|10|LicMat|
|56|lun 10/03/2025|2|PhyComp|1|E2B|
|57|mer 12/03/2025|2|RelAx|6|S4E|
|58|mer 12/03/2025|4|CryptoWithBlocks|6|S2B|
|59|lun 17/03/2025|2|PhyComp|2|E2B|
|60|mer 19/03/2025|2|RelAx|7|S4E|
|61|mer 19/03/2025|4|CryptoWithBlocks|7|S2B|
|62|ven 21/03/2025|5|RelAx|7|S5B+S5D|
|63|lun 24/03/2025|2|PhyComp|3|E2B|
|64|mer 26/03/2025|2|RelAx|8|S4E|
|65|lun 31/03/2025|2|PhyComp|4|E2B|
|66|mer 02/04/2025|2|RelAx|9|S4E|
|67|mer 02/04/2025|4|CryptoWithBlocks|9|S2B|
|68|ven 04/04/2025|5|RelAx|9|S5B+S5D|
|69|lun 07/04/2025|2|PhyComp|5|E2B|
|70|mer 09/04/2025|2|RelAx|10|S4E|
|71|mer 09/04/2025|4|CryptoWithBlocks|10|S2B|
|72|ven 11/04/2025|5|RelAx|10|S5B+S5D|
|73|lun 14/04/2025|2|PhyComp|6|E2B|
|74|mer 16/04/2025|4|CryptoWithBlocks|11|S2B|
|75|mer 23/04/2025|4|PhyComp|8|E2B|
|76|lun 28/04/2025|2|PhyComp|9|E2B|
|77|mer 30/04/2025|4|PhyComp|10|E2B|
|78|mer 07/05/2025|4|PhyComp|11|E2B|
|79|ven 09/05/2025|5|RelAx|11|S5B+S5D|
|80|ven 09/05/2025|6|RelAx|12|S5B+S5D|



## Pomili - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|
|1|ven 13/12/2024|5|RelAx|1|S5B+S5D|
|2|ven 20/12/2024|5|RelAx|2|S5B+S5D|
|3|ven 10/01/2025|5|RelAx|3|S5B+S5D|
|4|ven 17/01/2025|5|RelAx|4|S5B+S5D|
|5|ven 24/01/2025|5|RelAx|5|S5B+S5D|
|6|mer 05/02/2025|1|CryptoWithBlocks|1|S2B|
|7|mer 05/02/2025|2|RelAx|1|S4E|
|8|mer 05/02/2025|4|CryptoWithBlocks|2|S2B|
|9|mer 12/02/2025|2|RelAx|2|S4E|
|10|mer 12/02/2025|4|CryptoWithBlocks|3|S2B|
|11|mer 19/02/2025|2|RelAx|3|S4E|
|12|mer 19/02/2025|4|CryptoWithBlocks|4|S2B|
|13|mer 26/02/2025|2|RelAx|4|S4E|
|14|mer 05/03/2025|2|RelAx|5|S4E|
|15|mer 05/03/2025|4|CryptoWithBlocks|5|S2B|
|16|mer 12/03/2025|2|RelAx|6|S4E|
|17|mer 12/03/2025|4|CryptoWithBlocks|6|S2B|
|18|mer 19/03/2025|2|RelAx|7|S4E|
|19|mer 19/03/2025|4|CryptoWithBlocks|7|S2B|
|20|ven 21/03/2025|5|RelAx|7|S5B+S5D|
|21|mer 26/03/2025|2|RelAx|8|S4E|
|22|mer 02/04/2025|2|RelAx|9|S4E|
|23|mer 02/04/2025|4|CryptoWithBlocks|9|S2B|
|24|ven 04/04/2025|5|RelAx|9|S5B+S5D|
|25|mer 09/04/2025|2|RelAx|10|S4E|
|26|mer 09/04/2025|4|CryptoWithBlocks|10|S2B|
|27|ven 11/04/2025|5|RelAx|10|S5B+S5D|
|28|mer 16/04/2025|4|CryptoWithBlocks|11|S2B|
|29|ven 09/05/2025|5|RelAx|11|S5B+S5D|
|30|ven 09/05/2025|6|RelAx|12|S5B+S5D|



## Sciamanna - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|
|1|mer 11/12/2024|3|RelAx|1|S5E|
|2|mer 11/12/2024|4|RelAx|1|S4F|
|3|lun 16/12/2024|1|RelAx|2|S4F|
|4|lun 16/12/2024|2|RelAx|2|S5E|
|5|mer 18/12/2024|3|RelAx|3|S5E|
|6|mer 18/12/2024|4|RelAx|3|S4F|
|7|mer 08/01/2025|3|RelAx|4|S5E|
|8|mer 08/01/2025|4|RelAx|4|S4F|
|9|lun 13/01/2025|1|RelAx|5|S4F|
|10|lun 13/01/2025|2|RelAx|5|S5E|
|11|mer 15/01/2025|3|RelAx|6|S5E|
|12|mer 15/01/2025|4|RelAx|6|S4F|
|13|lun 20/01/2025|1|RelAx|7|S4F|
|14|lun 20/01/2025|2|RelAx|7|S5E|
|15|mer 22/01/2025|3|RelAx|8|S5E|
|16|mer 22/01/2025|4|RelAx|8|S4F|
|17|mer 29/01/2025|3|RelAx|9|S5E|
|18|mer 29/01/2025|4|RelAx|9|S4F|
|19|mer 29/01/2025|5|CyberSec|1|S3B|
|20|mer 29/01/2025|6|CyberSec|1|S4B|
|21|ven 31/01/2025|4|CyberSec|2|S4B|
|22|ven 31/01/2025|6|CyberSec|2|S3B|
|23|mer 05/02/2025|1|CryptoWithBlocks|1|S1B|
|24|mer 05/02/2025|3|RelAx|10|S5E|
|25|mer 05/02/2025|5|CyberSec|3|S3B|
|26|sab 08/02/2025|4|CryptoWithBlocks|2|S1B|
|27|mer 12/02/2025|1|CryptoWithBlocks|3|S1B|
|28|mer 12/02/2025|5|CyberSec|5|S3B|
|29|ven 14/02/2025|4|CyberSec|5|S4B|
|30|ven 14/02/2025|6|CyberSec|6|S3B|
|31|sab 15/02/2025|4|CryptoWithBlocks|4|S1B|
|32|mer 19/02/2025|1|CryptoWithBlocks|5|S1B|
|33|mer 19/02/2025|5|CyberSec|7|S3B|
|34|mer 19/02/2025|6|CyberSec|6|S4B|
|35|sab 22/02/2025|4|CryptoWithBlocks|6|S1B|
|36|mer 26/02/2025|4|RelAx|10|S4F|
|37|sab 01/03/2025|4|CryptoWithBlocks|8|S1B|
|38|mer 05/03/2025|1|CryptoWithBlocks|9|S1B|
|39|mer 05/03/2025|5|CyberSec|8|S3B|
|40|mer 05/03/2025|6|CyberSec|10|S4B|
|41|ven 07/03/2025|4|CyberSec|11|S4B|
|42|ven 07/03/2025|6|CyberSec|9|S3B|
|43|sab 08/03/2025|4|CryptoWithBlocks|10|S1B|
|44|lun 10/03/2025|2|PhyComp|1|E2B|
|45|mer 12/03/2025|1|CryptoWithBlocks|11|S1B|
|46|mer 12/03/2025|5|CyberSec|10|S3B|
|47|mer 12/03/2025|6|CyberSec|12|S4B|
|48|lun 17/03/2025|2|PhyComp|2|E2B|
|49|mer 19/03/2025|5|CyberSec|12|S3B|
|50|mer 19/03/2025|6|CyberSec|14|S4B|
|51|ven 21/03/2025|4|CyberSec|15|S4B|
|52|ven 21/03/2025|6|CyberSec|13|S3B|
|53|lun 24/03/2025|2|PhyComp|3|E2B|
|54|mer 26/03/2025|5|CyberSec|14|S3B|
|55|mer 26/03/2025|6|CyberSec|16|S4B|
|56|ven 28/03/2025|4|CyberSec|17|S4B|
|57|ven 28/03/2025|6|CyberSec|15|S3B|
|58|lun 31/03/2025|2|PhyComp|4|E2B|
|59|mer 02/04/2025|5|CyberSec|16|S3B|
|60|mer 02/04/2025|6|CyberSec|18|S4B|
|61|ven 04/04/2025|4|CyberSec|19|S4B|
|62|ven 04/04/2025|6|CyberSec|17|S3B|
|63|lun 07/04/2025|2|PhyComp|5|E2B|
|64|mer 09/04/2025|5|CyberSec|18|S3B|
|65|mer 09/04/2025|6|CyberSec|20|S4B|
|66|ven 11/04/2025|4|CyberSec|21|S4B|
|67|ven 11/04/2025|6|CyberSec|19|S3B|
|68|lun 14/04/2025|2|PhyComp|6|E2B|
|69|mer 16/04/2025|5|CyberSec|20|S3B|
|70|mer 16/04/2025|6|CyberSec|22|S4B|
|71|mer 23/04/2025|4|PhyComp|8|E2B|
|72|mer 23/04/2025|5|CyberSec|21|S3B|
|73|mer 23/04/2025|6|CyberSec|24|S4B|
|74|lun 28/04/2025|2|PhyComp|9|E2B|
|75|mer 30/04/2025|4|PhyComp|10|E2B|
|76|mer 30/04/2025|5|CyberSec|22|S3B|
|77|mer 30/04/2025|6|CyberSec|26|S4B|
|78|mer 07/05/2025|4|PhyComp|11|E2B|
|79|mer 07/05/2025|6|CyberSec|28|S4B|
|80|ven 09/05/2025|4|CyberSec|29|S4B|



## Taccari - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|
|1|mer 29/01/2025|5|CyberSec|1|S3B|
|2|mer 29/01/2025|6|CyberSec|1|S4B|
|3|ven 31/01/2025|4|CyberSec|2|S4B|
|4|ven 31/01/2025|6|CyberSec|2|S3B|
|5|mar 04/02/2025|2|CryptoWithBlocks|1|S4G|
|6|mer 05/02/2025|1|CryptoWithBlocks|1|S1B|
|7|mer 05/02/2025|5|CyberSec|3|S3B|
|8|sab 08/02/2025|4|CryptoWithBlocks|2|S1B|
|9|mar 11/02/2025|2|CryptoWithBlocks|2|S4G|
|10|mer 12/02/2025|1|CryptoWithBlocks|3|S1B|
|11|mer 12/02/2025|5|CyberSec|5|S3B|
|12|ven 14/02/2025|4|CyberSec|5|S4B|
|13|ven 14/02/2025|6|CyberSec|6|S3B|
|14|sab 15/02/2025|4|CryptoWithBlocks|4|S1B|
|15|mar 18/02/2025|2|CryptoWithBlocks|3|S4G|
|16|mer 19/02/2025|1|CryptoWithBlocks|5|S1B|
|17|mer 19/02/2025|5|CyberSec|7|S3B|
|18|mer 19/02/2025|6|CyberSec|6|S4B|
|19|sab 22/02/2025|4|CryptoWithBlocks|6|S1B|
|20|sab 01/03/2025|4|CryptoWithBlocks|8|S1B|
|21|mar 04/03/2025|2|CryptoWithBlocks|5|S4G|
|22|mer 05/03/2025|1|CryptoWithBlocks|9|S1B|
|23|mer 05/03/2025|5|CyberSec|8|S3B|
|24|mer 05/03/2025|6|CyberSec|10|S4B|
|25|ven 07/03/2025|4|CyberSec|11|S4B|
|26|ven 07/03/2025|6|CyberSec|9|S3B|
|27|sab 08/03/2025|4|CryptoWithBlocks|10|S1B|
|28|mar 11/03/2025|2|CryptoWithBlocks|6|S4G|
|29|mer 12/03/2025|1|CryptoWithBlocks|11|S1B|
|30|mer 12/03/2025|5|CyberSec|10|S3B|
|31|mer 12/03/2025|6|CyberSec|12|S4B|
|32|mar 18/03/2025|2|CryptoWithBlocks|7|S4G|
|33|mer 19/03/2025|5|CyberSec|12|S3B|
|34|mer 19/03/2025|6|CyberSec|14|S4B|
|35|ven 21/03/2025|4|CyberSec|15|S4B|
|36|ven 21/03/2025|6|CyberSec|13|S3B|
|37|mar 25/03/2025|2|CryptoWithBlocks|8|S4G|
|38|mer 26/03/2025|5|CyberSec|14|S3B|
|39|mer 26/03/2025|6|CyberSec|16|S4B|
|40|ven 28/03/2025|4|CyberSec|17|S4B|
|41|ven 28/03/2025|6|CyberSec|15|S3B|
|42|mar 01/04/2025|2|CryptoWithBlocks|9|S4G|
|43|mer 02/04/2025|5|CyberSec|16|S3B|
|44|mer 02/04/2025|6|CyberSec|18|S4B|
|45|ven 04/04/2025|4|CyberSec|19|S4B|
|46|ven 04/04/2025|6|CyberSec|17|S3B|
|47|mar 08/04/2025|2|CryptoWithBlocks|10|S4G|
|48|mer 09/04/2025|5|CyberSec|18|S3B|
|49|mer 09/04/2025|6|CyberSec|20|S4B|
|50|ven 11/04/2025|4|CyberSec|21|S4B|
|51|ven 11/04/2025|6|CyberSec|19|S3B|
|52|mar 15/04/2025|2|CryptoWithBlocks|11|S4G|
|53|mer 16/04/2025|5|CyberSec|20|S3B|
|54|mer 16/04/2025|6|CyberSec|22|S4B|
|55|mer 23/04/2025|5|CyberSec|21|S3B|
|56|mer 23/04/2025|6|CyberSec|24|S4B|
|57|mer 30/04/2025|5|CyberSec|22|S3B|
|58|mer 30/04/2025|6|CyberSec|26|S4B|
|59|mer 07/05/2025|6|CyberSec|28|S4B|
|60|ven 09/05/2025|4|CyberSec|29|S4B|





In [6]:
report.clear()
# Impegni giornalieri
report.addTitle("Orario del personale")
for p in ['Lorenzoni', 'Massi', 'Pomili', 'Sciamanna', 'Taccari']:
    schedRep = Report()
    schedRep.addSubTitle(f"{p} - Giorni")
    schedRep.addDailySchedule(p, con)
    schedRep.saveToFile("./gen_docs/schedule/" + p.lower() + ".md")
    report.append(schedRep.markdown())

report.display()

# Orario del personale

## Lorenzoni - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|
|1|mar 04/02/2025|2|CryptoWithBlocks|
|2|mar 11/02/2025|2|CryptoWithBlocks|
|3|mar 18/02/2025|2|CryptoWithBlocks|
|4|mar 04/03/2025|2|CryptoWithBlocks|
|5|mar 11/03/2025|2|CryptoWithBlocks|
|6|mar 18/03/2025|2|CryptoWithBlocks|
|7|mar 25/03/2025|2|CryptoWithBlocks|
|8|mar 01/04/2025|2|CryptoWithBlocks|
|9|mar 08/04/2025|2|CryptoWithBlocks|
|10|mar 15/04/2025|2|CryptoWithBlocks|



## Massi - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|
|1|mer 11/12/2024|3,4|RelAx|
|2|ven 13/12/2024|5|RelAx|
|3|lun 16/12/2024|1,2|RelAx|
|4|mer 18/12/2024|3,4|RelAx|
|5|ven 20/12/2024|5|RelAx|
|6|mer 08/01/2025|3,4|RelAx|
|7|ven 10/01/2025|5|RelAx|
|8|lun 13/01/2025|1,2|RelAx|
|9|mer 15/01/2025|3,4|RelAx|
|10|ven 17/01/2025|5|RelAx|
|11|lun 20/01/2025|1,2|RelAx|
|12|mer 22/01/2025|3,4|RelAx|
|13|ven 24/01/2025|5|RelAx|
|14|mer 29/01/2025|3,4|RelAx|
|15|ven 31/01/2025|5,6|LabMat2|
|16|lun 03/02/2025|1,2|RicOp|
|17|mer 05/02/2025|1,2,3,4|CryptoWithBlocks,RelAx|
|18|ven 07/02/2025|5,6|LabMat2|
|19|lun 10/02/2025|1,2|RicOp|
|20|mer 12/02/2025|2,4|CryptoWithBlocks,RelAx|
|21|ven 14/02/2025|5,6|LabMat2|
|22|lun 17/02/2025|1,2|RicOp|
|23|mer 19/02/2025|2,4|CryptoWithBlocks,RelAx|
|24|lun 24/02/2025|1,2|RicOp|
|25|mer 26/02/2025|2,4|RelAx|
|26|ven 28/02/2025|5,6|LabMat2|
|27|lun 03/03/2025|1,2|RicOp|
|28|mer 05/03/2025|2,4|CryptoWithBlocks,RelAx|
|29|ven 07/03/2025|5,6|LabMat2|
|30|lun 10/03/2025|2|PhyComp|
|31|mer 12/03/2025|2,4|CryptoWithBlocks,RelAx|
|32|lun 17/03/2025|2|PhyComp|
|33|mer 19/03/2025|2,4|CryptoWithBlocks,RelAx|
|34|ven 21/03/2025|5|RelAx|
|35|lun 24/03/2025|2|PhyComp|
|36|mer 26/03/2025|2|RelAx|
|37|lun 31/03/2025|2|PhyComp|
|38|mer 02/04/2025|2,4|CryptoWithBlocks,RelAx|
|39|ven 04/04/2025|5|RelAx|
|40|lun 07/04/2025|2|PhyComp|
|41|mer 09/04/2025|2,4|CryptoWithBlocks,RelAx|
|42|ven 11/04/2025|5|RelAx|
|43|lun 14/04/2025|2|PhyComp|
|44|mer 16/04/2025|4|CryptoWithBlocks|
|45|mer 23/04/2025|4|PhyComp|
|46|lun 28/04/2025|2|PhyComp|
|47|mer 30/04/2025|4|PhyComp|
|48|mer 07/05/2025|4|PhyComp|
|49|ven 09/05/2025|5,6|RelAx|



## Pomili - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|
|1|ven 13/12/2024|5|RelAx|
|2|ven 20/12/2024|5|RelAx|
|3|ven 10/01/2025|5|RelAx|
|4|ven 17/01/2025|5|RelAx|
|5|ven 24/01/2025|5|RelAx|
|6|mer 05/02/2025|1,2,4|CryptoWithBlocks,RelAx|
|7|mer 12/02/2025|2,4|CryptoWithBlocks,RelAx|
|8|mer 19/02/2025|2,4|CryptoWithBlocks,RelAx|
|9|mer 26/02/2025|2|RelAx|
|10|mer 05/03/2025|2,4|CryptoWithBlocks,RelAx|
|11|mer 12/03/2025|2,4|CryptoWithBlocks,RelAx|
|12|mer 19/03/2025|2,4|CryptoWithBlocks,RelAx|
|13|ven 21/03/2025|5|RelAx|
|14|mer 26/03/2025|2|RelAx|
|15|mer 02/04/2025|2,4|CryptoWithBlocks,RelAx|
|16|ven 04/04/2025|5|RelAx|
|17|mer 09/04/2025|2,4|CryptoWithBlocks,RelAx|
|18|ven 11/04/2025|5|RelAx|
|19|mer 16/04/2025|4|CryptoWithBlocks|
|20|ven 09/05/2025|5,6|RelAx|



## Sciamanna - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|
|1|mer 11/12/2024|3,4|RelAx|
|2|lun 16/12/2024|1,2|RelAx|
|3|mer 18/12/2024|3,4|RelAx|
|4|mer 08/01/2025|3,4|RelAx|
|5|lun 13/01/2025|1,2|RelAx|
|6|mer 15/01/2025|3,4|RelAx|
|7|lun 20/01/2025|1,2|RelAx|
|8|mer 22/01/2025|3,4|RelAx|
|9|mer 29/01/2025|3,4,5,6|CyberSec,RelAx|
|10|ven 31/01/2025|4,6|CyberSec|
|11|mer 05/02/2025|1,3,5|CryptoWithBlocks,CyberSec,RelAx|
|12|sab 08/02/2025|4|CryptoWithBlocks|
|13|mer 12/02/2025|1,5|CryptoWithBlocks,CyberSec|
|14|ven 14/02/2025|4,6|CyberSec|
|15|sab 15/02/2025|4|CryptoWithBlocks|
|16|mer 19/02/2025|1,5,6|CryptoWithBlocks,CyberSec|
|17|sab 22/02/2025|4|CryptoWithBlocks|
|18|mer 26/02/2025|4|RelAx|
|19|sab 01/03/2025|4|CryptoWithBlocks|
|20|mer 05/03/2025|1,5,6|CryptoWithBlocks,CyberSec|
|21|ven 07/03/2025|4,6|CyberSec|
|22|sab 08/03/2025|4|CryptoWithBlocks|
|23|lun 10/03/2025|2|PhyComp|
|24|mer 12/03/2025|1,5,6|CryptoWithBlocks,CyberSec|
|25|lun 17/03/2025|2|PhyComp|
|26|mer 19/03/2025|5,6|CyberSec|
|27|ven 21/03/2025|4,6|CyberSec|
|28|lun 24/03/2025|2|PhyComp|
|29|mer 26/03/2025|5,6|CyberSec|
|30|ven 28/03/2025|4,6|CyberSec|
|31|lun 31/03/2025|2|PhyComp|
|32|mer 02/04/2025|5,6|CyberSec|
|33|ven 04/04/2025|4,6|CyberSec|
|34|lun 07/04/2025|2|PhyComp|
|35|mer 09/04/2025|5,6|CyberSec|
|36|ven 11/04/2025|4,6|CyberSec|
|37|lun 14/04/2025|2|PhyComp|
|38|mer 16/04/2025|5,6|CyberSec|
|39|mer 23/04/2025|4,5,6|CyberSec,PhyComp|
|40|lun 28/04/2025|2|PhyComp|
|41|mer 30/04/2025|4,5,6|CyberSec,PhyComp|
|42|mer 07/05/2025|4,6|CyberSec,PhyComp|
|43|ven 09/05/2025|4|CyberSec|



## Taccari - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|
|1|mer 29/01/2025|5,6|CyberSec|
|2|ven 31/01/2025|4,6|CyberSec|
|3|mar 04/02/2025|2|CryptoWithBlocks|
|4|mer 05/02/2025|1,5|CryptoWithBlocks,CyberSec|
|5|sab 08/02/2025|4|CryptoWithBlocks|
|6|mar 11/02/2025|2|CryptoWithBlocks|
|7|mer 12/02/2025|1,5|CryptoWithBlocks,CyberSec|
|8|ven 14/02/2025|4,6|CyberSec|
|9|sab 15/02/2025|4|CryptoWithBlocks|
|10|mar 18/02/2025|2|CryptoWithBlocks|
|11|mer 19/02/2025|1,5,6|CryptoWithBlocks,CyberSec|
|12|sab 22/02/2025|4|CryptoWithBlocks|
|13|sab 01/03/2025|4|CryptoWithBlocks|
|14|mar 04/03/2025|2|CryptoWithBlocks|
|15|mer 05/03/2025|1,5,6|CryptoWithBlocks,CyberSec|
|16|ven 07/03/2025|4,6|CyberSec|
|17|sab 08/03/2025|4|CryptoWithBlocks|
|18|mar 11/03/2025|2|CryptoWithBlocks|
|19|mer 12/03/2025|1,5,6|CryptoWithBlocks,CyberSec|
|20|mar 18/03/2025|2|CryptoWithBlocks|
|21|mer 19/03/2025|5,6|CyberSec|
|22|ven 21/03/2025|4,6|CyberSec|
|23|mar 25/03/2025|2|CryptoWithBlocks|
|24|mer 26/03/2025|5,6|CyberSec|
|25|ven 28/03/2025|4,6|CyberSec|
|26|mar 01/04/2025|2|CryptoWithBlocks|
|27|mer 02/04/2025|5,6|CyberSec|
|28|ven 04/04/2025|4,6|CyberSec|
|29|mar 08/04/2025|2|CryptoWithBlocks|
|30|mer 09/04/2025|5,6|CyberSec|
|31|ven 11/04/2025|4,6|CyberSec|
|32|mar 15/04/2025|2|CryptoWithBlocks|
|33|mer 16/04/2025|5,6|CyberSec|
|34|mer 23/04/2025|5,6|CyberSec|
|35|mer 30/04/2025|5,6|CyberSec|
|36|mer 07/05/2025|6|CyberSec|
|37|ven 09/05/2025|4|CyberSec|





In [7]:
con.close()